<a href="https://colab.research.google.com/github/kkrree/todaysPython/blob/master/%ED%85%8D%EC%8A%A4%ED%8A%B8_%EB%B6%84%EC%84%9D_%EC%8B%A4%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 머신러닝 텍스트 분류실습
source : https://www.youtube.com/playlist?list=PLaTc2c6yEwmp35y9Z868VlV9NQpG-eBb6

머신러닝 기법 분류
![이미지](https://i.imgur.com/b6wcQ7f.png)

데이터 분석 및 머신러닝 적용 프로세스
![이미지](https://i.imgur.com/li0a2zj.png)



```
# 참고
scikit-learn 1.0 이상 버전 사용 권장 (sklearn__version__ 으로 확인)
1.0 이하의 버전이라면 conda prompt 혹은 터미널을 열어 아래 명령어로 업그레이드
conda : conda update scikit-learn
pip install scikit-learn --upgrade
```


### 데이터 셋 준비

In [2]:
#드라이브에서 파일 가져오기
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# 라이브러리 및 파일 불러오기
# 데이터 분석을 위한 pandas, 수치계산을 위한 numpy, 시각화를 위한 seaborn, matplotlib 을 로드
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

path = "/content/drive/MyDrive/Nexon_Assignment/"
content =pd.read_csv(path + "content.csv")
comment =pd.read_csv(path + "comment.csv")

### 텍스트 데이터 수치 형태로 변경

In [4]:
# sklearn.feature_extraction.text의 CountVectorizer 로 BOW 벡터화
# fit_transform 으로 훈련 및 변환
from sklearn.feature_extraction.text import CountVectorizer 
cvect = CountVectorizer()
dtm = cvect.fit_transform(content['title']) 
dtm # sparse matrix 희소행렬로 변환된 것 확인

<6758x12170 sparse matrix of type '<class 'numpy.int64'>'
	with 41525 stored elements in Compressed Sparse Row format>

### DTM(document-term matrix)

In [5]:
# 피처 가져오기
featureNames = cvect.get_feature_names_out()
featureNames

array(['000', '00001', '00007프로를', ..., '𝗖𝗨𝗣', '𝗙𝗜𝗙𝗔', '𝗪𝗢𝗥𝗟𝗗'],
      dtype=object)

In [6]:
# 단어 사전 보기
cvect.vocabulary_

{'300억': 740,
 '구독자': 3358,
 '요청': 8643,
 '1위': 367,
 '스쿼드': 7338,
 '신규': 7525,
 'ln시즌': 1867,
 '포함': 11255,
 '잉글랜드': 9341,
 '완성': 8585,
 '피파4': 11417,
 '22ty': 551,
 '프랑스': 11323,
 'st': 2177,
 '킬리안': 10767,
 '음바페': 8922,
 '22toty': 548,
 '1카': 412,
 '리뷰와': 5094,
 '골모음': 3209,
 'fifa4': 1675,
 'france': 1689,
 'kyl': 1821,
 '떴다': 4836,
 '추석버닝': 10434,
 '강화데이': 2843,
 '금카까지': 3546,
 '가능해': 2648,
 '선수들': 6969,
 '떡상한': 4811,
 '상황': 6868,
 '빠칭코': 6531,
 '송편': 7181,
 '돌림빵': 4526,
 'ㄱㄱ': 2564,
 '지금': 9980,
 '하면': 11565,
 '좋은': 9821,
 '3가지': 794,
 'ln호나우두': 1884,
 '리뷰': 5091,
 '시세가': 7439,
 '갑자기': 2785,
 '개떡상한': 2926,
 '이유': 9133,
 '본캐': 6313,
 '21챔스': 521,
 '뤼디거': 5072,
 '금카': 3540,
 '샀다': 6797,
 '독일': 4496,
 '종결': 9774,
 '08맨유': 19,
 '리빌딩': 5097,
 '보면': 6244,
 '갬성에': 3019,
 '바로': 5879,
 '지려버리는': 10006,
 '신상': 7557,
 '듀오': 4636,
 '영입': 8365,
 '출시': 10481,
 '했는데': 11832,
 '이것좀': 8983,
 '보세요': 6253,
 '이렇게도': 9054,
 '가능': 2633,
 '빠찡코': 6530,
 '풀리고': 11290,
 '시세': 7438,
 '난리났네요': 3897,
 'vip멤버쉽

In [7]:
# dtm(document-term matrix)
pd.DataFrame(dtm.toarray(), columns = featureNames)

,000,00001,00007프로를,000냥,000억에,000억인가요,00시가,00시에,01,02,...,힐플릭,힘ㅋㅋ,힘내보자,힘드신,힘든,힘들걸,힘을,𝗖𝗨𝗣,𝗙𝗜𝗙𝗔,𝗪𝗢𝗥𝗟𝗗
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6753,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6754,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6755,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6756,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 문제와 정답 만들기

In [8]:
# 독랍변수로 사용할 x 변수에 dtm array를 할당
X = dtm.toarray()
X

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [9]:
# 종속변수로 사용할 변수에 정답값인  값 할당
Y = content['viewCount']
Y

0       106176
1         1371
2        60017
3           27
4        92075
         ...  
6753         4
6754         7
6755      6388
6756     16868
6757       780
Name: viewCount, Length: 6758, dtype: int64

### 데이터셋 나누기
* 현재 예제에서는 임의로 나눔
* test_train_split 을 활용하여 나눌 수 있음

In [10]:
# 8:2 의 비율로 구하기 위해 전체 데이터의 행에서 80% 위치에 해당되는 값을 구해서 split_count 라는 변수에 담습니다.
splitCount = int(content.shape[0]*0.8)
splitCount

5406

In [11]:
# 학습 세트 만들기
X_train = X[:splitCount]
Y_train = Y[:splitCount]
print(X_train.shape, Y_train.shape) # 행 갯수 같은지 확인
X_train, Y_train

(5406, 12170) (5406,)


(array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]), 0       106176
 1         1371
 2        60017
 3           27
 4        92075
          ...  
 5401      2336
 5402        21
 5403        82
 5404      1352
 5405       482
 Name: viewCount, Length: 5406, dtype: int64)

In [12]:
# 예측 세트 만들기
X_test = X[splitCount:]
Y_test = Y[splitCount:]
print(X_test.shape, Y_test.shape) # 행 갯수 같은지 확인
X_test, Y_test

(1352, 12170) (1352,)


(array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]), 5406      169
 5407    25739
 5408      347
 5409    13804
 5410    34444
         ...  
 6753        4
 6754        7
 6755     6388
 6756    16868
 6757      780
 Name: viewCount, Length: 1352, dtype: int64)